In [59]:
import sys
sys.path.append('/home/grant/workdir/build/ssl-sky-surveys/')

import torch
import numpy as np
import matplotlib.pyplot as plt
import h5py
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

from utils.load_trained_model import load_model_from_checkpoint
%matplotlib inline
from astropy.visualization import make_lupton_rgb
import torchvision.transforms as transforms
import torchvision.models as models


In [60]:
import time

### Load model

In [61]:
ckpt = '/external/grant/models/moco/DR2/custom50jc_stiff/checkpoint_0199.pth.tar'
model = load_model_from_checkpoint(ckpt,basenet=False,num_channels=3,mlp=True)

INFO:root:Loading checkpoint /external/grant/models/moco/DR2/custom50jc_stiff/checkpoint_0199.pth.tar
INFO:root:Chekpoint loaded. Checkpoint epoch 200


### Load data

In [23]:
#data = h5py.File('/external/grant/data/sdss_w_specz_valid.h5', 'r')
data = np.load('/external/grant/data/DES_tiles_imarray.npy')
data = data.reshape(data.shape[0],3,228,228)

In [14]:
#print("Data keys:", data.keys())
#print("Images shape:", data['images'].shape)

In [24]:

print(data.shape)
#imt = transforms.ToTensor()(img).unsqueeze_(0)

(2542, 3, 228, 228)


### Inference

In [55]:
t0 = time.time()
with torch.no_grad():
    #datai = torch.from_numpy(data[0:100])
    #datai = torch.from_numpy(data[0:20]) 
    #reps = model(datai).cpu().detach().numpy()

    datai = torch.from_numpy(np.flip(data[0:100],axis=(2,3)).copy())
    reps = model(datai.clone().detach()).cpu().detach().numpy()
    
print(time.time()-t0)

58.83113646507263


In [47]:
reps.shape


(100, 128)

In [56]:
with torch.no_grad():
    for i in range(1,10):
        #datai = torch.from_numpy(data[100*i:100*(i+1)].copy())        
        datai = torch.from_numpy(np.flip(data[100*i:100*(i+1)],axis=(2,3)).copy())
        #repsi = model(datai).cpu().detach().numpy()
        repsi = model(datai.clone().detach()).cpu().detach().numpy()
        reps = np.concatenate((reps,repsi))

In [43]:
#with torch.no_grad():
#    datai = torch.from_numpy(data[2500:].copy())        
#    #datai = torch.from_numpy(np.flip(data[100*i:100*(i+1)],axis=(2,3)).copy())
#    repsi = model(datai).cpu().detach().numpy()
#    #repsi = model(torch.tensor(data['images'][200*i:200*(i+1)][:,:,::-1])).cpu().detach().numpy()
#    reps = np.concatenate((reps,repsi))

In [57]:
reps.shape

(1000, 128)

In [58]:
np.save('/external/grant/models/moco/DR2/custom50jc_stiff/des_encoded_flip_2.npy',reps)

In [38]:
e = np.load('/external/grant/data/moco/DES/des_encoded.npy')
print(e.shape)

(1000, 2048)


In [18]:
res50 = models.resnet50
print(res50())

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, trac

In [4]:
from collections import OrderedDict


In [7]:
checkpoint = torch.load(ckpt, map_location='cpu')
model_key = 'model_state' if 'model_state' in checkpoint else 'state_dict'

for key in checkpoint[model_key].keys():
    print(key)

module.queue
module.queue_ptr
module.encoder_q.conv1.weight
module.encoder_q.bn1.weight
module.encoder_q.bn1.bias
module.encoder_q.bn1.running_mean
module.encoder_q.bn1.running_var
module.encoder_q.bn1.num_batches_tracked
module.encoder_q.layer1.0.conv1.weight
module.encoder_q.layer1.0.bn1.weight
module.encoder_q.layer1.0.bn1.bias
module.encoder_q.layer1.0.bn1.running_mean
module.encoder_q.layer1.0.bn1.running_var
module.encoder_q.layer1.0.bn1.num_batches_tracked
module.encoder_q.layer1.0.conv2.weight
module.encoder_q.layer1.0.bn2.weight
module.encoder_q.layer1.0.bn2.bias
module.encoder_q.layer1.0.bn2.running_mean
module.encoder_q.layer1.0.bn2.running_var
module.encoder_q.layer1.0.bn2.num_batches_tracked
module.encoder_q.layer1.0.conv3.weight
module.encoder_q.layer1.0.bn3.weight
module.encoder_q.layer1.0.bn3.bias
module.encoder_q.layer1.0.bn3.running_mean
module.encoder_q.layer1.0.bn3.running_var
module.encoder_q.layer1.0.bn3.num_batches_tracked
module.encoder_q.layer1.0.downsample.0.w

In [9]:
# some checkpoints have a different name for the model key

new_model_state = OrderedDict()
for key in checkpoint[model_key].keys():
    if 'encoder' in key:
        if 'encoder_q' in key:
            name = str(key).replace('module.encoder_q.', '')
            new_model_state[name] = checkpoint[model_key][key]
    elif 'module.' in key:
        name = str(key).replace('module.', '')
        new_model_state[name] = checkpoint[model_key][key]

msg = model.load_state_dict(new_model_state, strict=False)
if msg.missing_keys == ['fc.weight', 'fc.bias']:
    print('true')

queue
queue_ptr


In [11]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, trac

In [7]:
msg = model.load_state_dict(new_model_state, strict=False)
print(msg)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['queue', 'queue_ptr'])


In [27]:
import torchvision.models as basemodels

model = basemodels.resnet50().to('cpu')

In [62]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, trac

In [35]:
test = np.load('/external/grant/models/moco/custom50jc/des_mlp_encoded.npy')
print(test.shape)

(5961, 128)


(5961, 2048)
